<h1> Advanced Learning for Text and Graph Data </h1>
<b> Université Paris-Saclay - Master M2 Data Science - February/March 2017</b> <br>
<i> Students : Peter Martigny & Mehdi Miah </i> <br>

# Third part  : predict and measure the error

## Open the data

In [1]:
#our own modules
from map_tools import * #functions computing mean average precision
from handy_structures import *

#common libraries
import pandas as pd
import numpy as np
#from tqdm import *
import random
import pandas as pd
from sklearn.model_selection import train_test_split

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
#path to data and intermediate results
path_to_data = "..\\data\\"
path_to_results = "..\\results\\"

## == open files ====

#original data
training_set = pd.read_csv(path_to_data + 'training_set.csv')
training_info = pd.read_csv(path_to_data + 'training_info.csv', 
                            dtype = {'mid': object, 'date': object, 'body': object, 'recipients' : object})
test_set = pd.read_csv(path_to_data + 'test_set.csv')
test_info = pd.read_csv(path_to_data + 'test_info.csv',
                        dtype = {'mid': object, 'date': object, 'body': object, 'recipients' : object})

#intermediate data
training_df = pd.read_csv(path_to_results + 'training_df.csv',
                          dtype = {'mid': object, 'sender': object, 'date': object, 'body': object, 'recipients' : object})
test_df = pd.read_csv(path_to_results + 'test_df.csv',
                          dtype = {'mid': object, 'sender': object, 'date': object, 'body': object})

In [3]:
training_set.head()

,sender,mids
0,karen.buckley@enron.com,158713 158697 200301 158679 278595 298162 2002...
1,amr.ibrahim@enron.com,215241 3437 215640 3506 191790 3517 3520 3562 ...
2,andrea.ring@enron.com,270705 270706 270707 270708 270709 270710 2707...
3,sylvia.hu@enron.com,111444 111422 183084 111412 111347 110883 1105...
4,phillip.platter@enron.com,327074 327384 327385 264443 274124 274125 2741...


In [4]:
training_info.head()

,mid,date,body,recipients
0,60,2000-07-25 08:14:00,Legal has been assessing the risks of doing bl...,robert.badeer@enron.com murray.o neil@enron.co...
1,66,2000-08-03 02:56:00,Attached is a spreadsheet to estimate export f...,kim.ward@enron.com robert.badeer@enron.com mur...
2,74,2000-08-15 05:37:00,Kevin/Bob: Here is a quick rundown on the cons...,robert.badeer@enron.com john.massey@enron.com ...
3,80,2000-08-20 14:12:00,check this out and let everyone know what s up...,robert.badeer@enron.com jeff.richter@enron.com
4,83,2000-08-22 08:17:00,Further to your letter to us (addressed to Mr....,pgillman@schiffhardin.com kamarlantes@calpx.co...


In [5]:
test_set.head()

,sender,mids
0,karen.buckley@enron.com,298389 332383 298390 284071 366982 81773 81791...
1,amr.ibrahim@enron.com,48260 48465 50344 48268 50330 48237 189979 189...
2,andrea.ring@enron.com,366364 271168 271172 271167 271189
3,sylvia.hu@enron.com,134931 134856 233549 233517 134895 233584 3736...
4,phillip.platter@enron.com,274220 274225 274215 274223 274214 274207 2742...


In [6]:
test_info.head()

,mid,date,body
0,1577,2001-11-19 06:59:51,Note: Stocks of heating oil are very high for...
1,1750,2002-03-05 08:46:57,"Kevin Hyatt and I are going for ""sghetti"" at S..."
2,1916,2002-02-13 14:17:39,This was forwarded to me and it is funny. - Wi...
3,2094,2002-01-22 11:33:56,I will be in to and happy to assist too. I ma...
4,2205,2002-01-11 07:12:19,Thanks. I needed a morning chuckle.


In [7]:
training_df.head()

,mid,sender,date,body,recipients
0,47361,enron_update@concureworkplace.com,0001-08-26 22:16:36,The following reports have been waiting for yo...,kimberly.watson@enron.com
1,47362,enron_update@concureworkplace.com,0001-08-27 22:21:02,The following reports have been waiting for yo...,kimberly.watson@enron.com
2,47363,enron_update@concureworkplace.com,0001-08-28 22:25:35,The following reports have been waiting for yo...,kimberly.watson@enron.com
3,45909,enron_update@concureworkplace.com,0001-09-13 22:24:08,Employee Name: Kimberly WatsonReport Name: E...,kimberly.watson@enron.com
4,82030,enron_update@concureworkplace.com,0001-09-17 09:24:00,The following expense report is ready for appr...,barry.tycholiz@enron.com


In [8]:
test_df.head()

,mid,sender,date,body
0,284098,jonathan.mckay@enron.com,2001-11-02 05:25:29,"How is everyone.....mother, child.........fath..."
1,272008,dutch.quigley@enron.com,2001-11-02 05:34:55,-----Original Message-----From: \tWesner-Soon...
2,49273,james.d.steffes@enron.com,2001-11-02 05:57:55,Janine -Ok for you to cover the whole country....
3,71901,kim.ward@enron.com,2001-11-02 06:10:47,when?
4,82354,barry.tycholiz@enron.com,2001-11-02 06:17:44,WOW.... I am positive that your beautiful wife...


# Cross validation

### First way : temporal on all emails

In [10]:
if False :
    #Temporal separation between training and test
    # already time ordered
    pct_train = 0.5
    idx_separation = round(pct_train * training_info.shape[0])
    mid_train = training_info['mid'][:idx_separation]
    mid_test = training_info['mid'][idx_separation:]

    # Separation in training_info
    training_info_train = training_info[:idx_separation]
    training_info_test = training_info[idx_separation:]

    #Separation in training_set
    training_set_train = training_set.copy()
    training_set_test = training_set.copy()

    for row in range(training_set_train.shape[0]):
        mids = training_set_train['mids'][row].split(' ')
        new_mid = list(set(mids) & set(mid_train))
        training_set_train['mids'][row] = ' '.join(new_mid)

    for row in range(training_set_test.shape[0]):
        mids = training_set_test['mids'][row].split(' ')
        new_mid = list(set(mids) & set(mid_test))
        training_set_test['mids'][row] = ' '.join(new_mid)      

In [11]:
if False:
    #dataframe
    training_df_train = transform_dataset(training_info_train, training_set_train)
    training_df_test = transform_dataset(training_info_test, training_set_test)

Some users are in the test set and NOT in the train set : mike.carson@enron.com, vkaminski@aol.com and christina.valdez@enron.com !


### Second way : pure randomness

In [11]:
if True:
    training_all_senders = training_set['sender'].get_values()

In [9]:
if True : 
    # Separation in training_df
    training_df_train, training_df_test = train_test_split(training_df, test_size=0.2, random_state=0)

    training_df_train = training_df_train.reset_index(drop=True)
    training_df_test = training_df_test.reset_index(drop=True)

    mid_train = training_df_train['mid']
    mid_test = training_df_test['mid']

In [12]:
if True:
    # Construction of _set with for each sender, the list of emails

    #Training
    training_set_train = pd.DataFrame()
    for sender in tqdm(training_all_senders):
        mids = training_df_train[training_df_train['sender'] == sender]['mid'].values.tolist()
        mids = ' '.join(list(map(str, mids)))
        training_set_train_sender = pd.DataFrame({'sender':[sender], 'mids': [mids]})

        training_set_train = training_set_train.append(training_set_train_sender)

    #reorder the columns
    training_set_train = training_set_train.reindex_axis(['sender', 'mids'], axis = 1)

    #add clean index
    training_set_train = training_set_train.reset_index(drop=True)

    #Test
    training_set_test = pd.DataFrame()
    for sender in tqdm(training_all_senders):
        mids = training_df_test[training_df_test['sender'] == sender]['mid'].values.tolist()
        mids = ' '.join(list(map(str, mids)))
        training_set_test_sender = pd.DataFrame({'sender':[sender], 'mids': [mids]})

        training_set_test = training_set_test.append(training_set_test_sender)

    #reorder the columns
    training_set_test = training_set_test.reindex_axis(['sender', 'mids'], axis = 1)

    #add clean index
    training_set_test = training_set_test.reset_index(drop=True)

In [18]:
if True:
    # Construction of _info with info about each mail

    #Training
    mids_train = training_df_train.mid.get_values().astype('str')
    training_info_train = training_info[(training_info['mid']).isin(mids_train)]

    #Test
    mids_test = training_df_test.mid.get_values().astype('str')
    training_info_test = training_info[(training_info['mid']).isin(mids_test)]

### third way : temporal by user

In [17]:
if False:
    training_all_senders = training_set['sender'].get_values()

In [19]:
if False:
    def split_data_v3(training_df, training_all_senders, pct_train = 0.8):
        #Initialisation of the two dataframe for learning and testing
        training_df_train = pd.DataFrame()
        training_df_test = pd.DataFrame()

        #We fill the values by taking the oldest emails in the train and the newest in test dataframe
        for sender in tqdm(training_all_senders):
            #dataframe with all emails sent by the sender
            training_df_sender = training_df[training_df['sender'] == sender] #non void
            limit = round(pct_train * training_df_sender.shape[0])

            training_df_train_sender = training_df_sender[:limit]
            training_df_test_sender = training_df_sender[limit:]

            training_df_train = training_df_train.append(training_df_train_sender)
            training_df_test = training_df_test.append(training_df_test_sender)

        #put the right index
        training_df_train = training_df_train.reset_index(drop=True)
        training_df_test = training_df_test.reset_index(drop=True)

        return training_df_train, training_df_test

    training_df_train, training_df_test = split_data_v3(training_df, training_all_senders, pct_train = 0.8)

Time to generate _info and _set dataframes.

In [20]:
if False:
    # Construction of _set with for each sender, the list of emails

    #Training
    training_set_train = pd.DataFrame()
    for sender in tqdm(training_all_senders):
        mids = training_df_train[training_df_train['sender'] == sender]['mid'].values.tolist()
        mids = ' '.join(list(map(str, mids)))
        training_set_train_sender = pd.DataFrame({'sender':[sender], 'mids': [mids]})

        training_set_train = training_set_train.append(training_set_train_sender)

    #reorder the columns
    training_set_train = training_set_train.reindex_axis(['sender', 'mids'], axis = 1)

    #add clean index
    training_set_train = training_set_train.reset_index(drop=True)

    #Test
    training_set_test = pd.DataFrame()
    for sender in tqdm(training_all_senders):
        mids = training_df_test[training_df_test['sender'] == sender]['mid'].values.tolist()
        mids = ' '.join(list(map(str, mids)))
        training_set_test_sender = pd.DataFrame({'sender':[sender], 'mids': [mids]})

        training_set_test = training_set_test.append(training_set_test_sender)

    #reorder the columns
    training_set_test = training_set_test.reindex_axis(['sender', 'mids'], axis = 1)

    #add clean index
    training_set_test = training_set_test.reset_index(drop=True)

In [22]:
training_set_train.head()

,sender,mids
0,karen.buckley@enron.com,390472 298372 158472 158594 365523 366836 1597...
1,amr.ibrahim@enron.com,191629 3437 3664 191797 215241 50792 190011 18...
2,andrea.ring@enron.com,270813 365818 270708 270747 270769 270768 2707...
3,sylvia.hu@enron.com,37547 35813 38516 38487 111444 108576 106923 2...
4,phillip.platter@enron.com,274158 274121 274186 274134 274162 274135 2741...


In [21]:
if False:
    # Construction of _info with info about each mail

    #Training
    mids_train = training_df_train.mid.get_values().astype('str')
    training_info_train = training_info[(training_info['mid']).isin(mids_train)]

    #Test
    mids_test = training_df_test.mid.get_values().astype('str')
    training_info_test = training_info[(training_info['mid']).isin(mids_test)]

In [23]:
training_info_train.head()

,mid,date,body,recipients
0,337707,2000-02-29 08:19:00,"Not yet, still trying to be creative in what I...",clayton.vernon@enron.com
1,299551,2001-04-02 13:07:00,I have just been informed that our P&L was ove...,john.lavorato@enron.com louise.kitchen@enron.com
2,368521,2001-04-11 04:13:00,Here is another one...---------------------- F...,bob.hall@enron.com
3,182846,1999-07-23 10:57:00,Did you receive my voice mail yesterday about ...,rod.nelson@enron.com
4,276472,2001-03-26 09:01:00,"Dad,Attached is Brett s baseball schedule. I ...",cneal@bluegate.com


### Sanity check on cross validation

Let's make some sanity test : 

In [19]:
## == Check on the number of email in each part ====
n_senders = 125

def sanity_check_n_emails(dataset_df, dataset_info, dataset_set, name):
    n_mails_dataset_df = dataset_df.shape[0]
    n_mails_dataset_info = dataset_info.shape[0]
    n_mails_dataset_set = np.sum([len(dataset_set.mids[i].split(' ')) for i in range(n_senders)])
    print('SANITY CHECK ON ' + name)
    print('Number of emails in the full dataframe : %.0f \t _info : %.0f \t _set : %.0f \n' 
          %(n_mails_dataset_df, n_mails_dataset_info, n_mails_dataset_set))

#In the training (train + test)
sanity_check_n_emails(training_df, training_info, training_set, "training")

#In the train
sanity_check_n_emails(training_df_train, training_info_train ,training_set_train , "training for train")

#In the test
sanity_check_n_emails(training_df_test, training_info_test ,training_set_test, "training for test")


SANITY CHECK ON training
Number of emails in the full dataframe : 43613 	 _info : 43613 	 _set : 43613 

SANITY CHECK ON training for train
Number of emails in the full dataframe : 34890 	 _info : 34890 	 _set : 34890 

SANITY CHECK ON training for test
Number of emails in the full dataframe : 8723 	 _info : 8723 	 _set : 8723 



# Some handy structures on the train of cv

In [20]:
address_books_train = get_address_books(training_info_train, training_set_train)

In [21]:
# save all unique sender names
emails_ids_per_sender_train = get_mids_per_sender(training_set_train)
all_senders_train = emails_ids_per_sender_train.keys()

# save all unique recipient names    
all_recs_train = list(set([elt[0] for sublist in address_books_train.values() for elt in sublist]))

# save all unique user names 
all_users_train = []
all_users_train.extend(all_senders_train)
all_users_train.extend(all_recs_train)
all_users_train = list(set(all_users_train))

In [27]:
if False:
    def get_mid_sender_recipient(dataset_df, sender, recipient):

        mask_sender = (sender == dataset_df.sender)
        mask_recipient = [recipient in dataset_df.recipients[i] for i in range(dataset_df.shape[0])]

        mask = mask_sender & mask_recipient

        return dataset_df['mid'][mask]


# Prédiction

## Random predictions on all users

In [22]:
# HERE THE PREDICTION WILL ONLY DEPENDS ON THE SENDER !
def predictions_random_per_sender(dataset_set, all_users, k=10):

    # will contain email ids, predictions for random baseline
    predictions_random_per_sender = {}

    for index, row in dataset_set.iterrows():
        name_ids = row.tolist() #sender + all ones mails)
        sender = name_ids[0]

        predictions_random_per_sender[sender] = random.sample(all_users, k)
        
    return predictions_random_per_sender    

In [24]:
#Make some predictions
predictions_random = predictions_random_per_sender(training_set_train, all_users_train)

#Fill the dataframe
training_df_test['random_pred'] = None
for row in range(training_df_test.shape[0]): #for each mail
    sender = training_df_test['sender'][row]

    #random prediction
    training_df_test['random_pred'][row] = predictions_random[sender] 

In [31]:
training_df_test.head()

,mid,sender,date,body,recipients,random_pred
0,27781,enron_update@concureworkplace.com,0001-10-04 08:34:46,The following expense report is ready for appr...,mark.whitt@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca..."
1,201622,enron_update@concureworkplace.com,0001-10-15 15:18:32,The Payment status has changed on the followin...,mark.mcconnell@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca..."
2,201623,enron_update@concureworkplace.com,0001-10-15 15:18:32,The Approval status has changed on the followi...,mark.mcconnell@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca..."
3,201624,enron_update@concureworkplace.com,0001-10-15 15:18:35,The Approval status has changed on the followi...,mark.mcconnell@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca..."
4,82795,enron_update@concureworkplace.com,0001-10-17 13:06:55,The following expense report is ready for appr...,barry.tycholiz@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca..."


## Random predictions on the address book

In [32]:
# HERE THE PREDICTION WILL ONLY DEPENDS ON THE SENDER !

def predictions_random_address_book_per_sender(dataset_set, address_books, k=10):
    '''
    Recommends recipients randomly amongst the addresss books of the sender
    The number of recipients may be under k if the sender has not sent enough emails.
    '''
    # will contain email ids, predictions for random baseline
    predictions_random_address_book_per_sender = {}
    for index, row in dataset_set.iterrows():
        name_ids = row.tolist() #sender + all ones mails)
        sender = name_ids[0]

        recipients_of_the_sender = [elt[0] for elt in address_books[sender]]
        n_recipients_of_the_sender = len(recipients_of_the_sender)
        
        predictions_random_address_book_per_sender[sender] = random.sample(recipients_of_the_sender, min(k,n_recipients_of_the_sender))
    return predictions_random_address_book_per_sender    

In [34]:
#Make some predictions
predictions_random_address_book = predictions_random_address_book_per_sender(training_set_train, address_books_train)

#Fill the dataframe
training_df_test['random_pred_address_book'] = None
for row in range(training_df_test.shape[0]): #for each mail
    sender = training_df_test['sender'][row]

    #random prediction
    training_df_test['random_pred_address_book'][row] = predictions_random_address_book[sender]    

## Frequency based predictions

In [25]:
# HERE THE PREDICTION WILL ONLY DEPENDS ON THE SENDER !
def predictions_frequency_per_sender(dataset_set, address_books, k = 10):

    # will contain email ids, predictions for frequency baseline
    predictions_frequency_per_sender = {}

    for index, row in dataset_set.iterrows():
        name_ids = row.tolist() #sender + all ones mails
        sender = name_ids[0]

        # select k most frequent recipients for the user
        predictions_frequency_per_sender[sender] = [elt[0] for elt in address_books[sender][:k]]
        
    return predictions_frequency_per_sender

In [27]:
#Make some predictions
predictions_frequency = predictions_frequency_per_sender(training_set_train, address_books_train)

#Fill the dataframe
training_df_test['freq_pred'] = None
for row in range(training_df_test.shape[0]): #for each mail
    sender = training_df_test['sender'][row]

    #random prediction
    training_df_test['freq_pred'][row] = predictions_frequency[sender]    

## Weighted frequency-based predictions

In [56]:
def get_weighted_score(dataset_df):
    '''
    Computes a score between the sender and the recipient
    score(sender, recipient) = sum_{emails from sender to recipients} 1/(number of recipients in the email)
    
    == Input ==
    a dataframe with all the data
    == Output ==
    a dictionnary where the key is the sender and the value the tuple (recipient, score)
    '''
    #Learning phase
    dict_weight = {} #the key is (sender, recipient)
    for row in tqdm(range(dataset_df.shape[0])):

        sender = dataset_df['sender'][row]
        recipients = dataset_df['recipients'][row].split(' ')
        n_recipients = len(recipients)

        for rec in recipients: #for each recipients
            key = sender, rec
            if key in dict_weight : 
                dict_weight[key] =  dict_weight[key] + 1./n_recipients
            else:
                dict_weight[key] = 1./n_recipients

    #The right order
    weight_sender_recipient = {} #the key is (sender)
    for key, value in dict_weight.items():
        sender = key[0]
        rec = key[1]

        if sender in weight_sender_recipient:
            weight_sender_recipient[sender] = sorted(weight_sender_recipient[sender] + [(rec,value)], key=operator.itemgetter(1), reverse = True)      
        else:
            weight_sender_recipient[sender] = [(rec,value)]
            
    return dict_weight, weight_sender_recipient      

In [57]:
dict_weight, weight_sender_recipient = get_weighted_score(training_df_train)

In [60]:
#Make some predictions
training_df_test['weighted_freq_pred'] = None
k=10

#Testing phase
for row in range(training_df_test.shape[0]): #for each mail
    sender = training_df_test['sender'][row]

    #random prediction
    training_df_test['weighted_freq_pred'][row] = [elt[0] for elt in weight_sender_recipient[sender][:k]]

## Looks for a name

In [74]:
def get_recipients_from_name(name, all_users):
    '''
    Gives 0 or 1 recipient email address whose name is the one given is the input
    
    == Input ==
    name : a string with the name one is looking for
    all_users : a list of names, where the name is looked for, ordered by preference / score
    == Output ==
    if name is found, an email address
    '''
    
    if((len(name) > 3)): #remove 'it'
        for rec in all_users : 
            rec_name = rec.split('@')[0] #the part before the @
            score = str.find(rec_name, name.lower()) #returns -1 if the name is not in the email address, otherwise its position
            if score > -1: #a corresponding email address has been found
                return [rec]
        return [] #no corresponding email address
    return [] #the given name is too short 

In [71]:
get_recipients_from_name('Mark', all_users_train)

['mark.fischer@enron.com']

In [77]:
#Make some predictions
training_df_test['name_freq_pred'] = None
k=10

#Testing phase
for row in tqdm(range(training_df_test.shape[0])): #for each mail
    sender = training_df_test['sender'][row]
    name_rec = training_df_test['body'][row].split(' ')[:1][0]

    #remove some nasty character
    #name_rec = str(name_rec).replace(',', '') 
    
    #find the possible recipients from this first word, amongst those the sender has already sent an email
    address_book_sender = [elt[0] for elt in address_books_train[sender]] #ordered by the frequency ! 
    
    recipients_from_name = get_recipients_from_name(name_rec, address_book_sender)

    #add the frequent recipients
    recipients_from_frequency = predictions_frequency[sender]
    
    #cumul
    result = recipients_from_name
    result.extend(x for x in recipients_from_frequency if x not in result)

    training_df_test['name_freq_pred'][row] = result[:k]


In [42]:
training_df_test.head()

,mid,sender,date,body,recipients,random_pred,random_pred_address_book,freq_pred,weighted_freq_pred,name_freq_pred
0,27781,enron_update@concureworkplace.com,0001-10-04 08:34:46,The following expense report is ready for appr...,mark.whitt@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca...","[joe.stepenovitch@enron.com, tmartin@enron.com...","[richard.shapiro@enron.com, rick.buy@enron.com...","[n, o, r, e, ., c, m, @, a, i]","[richard.shapiro@enron.com, rick.buy@enron.com..."
1,201622,enron_update@concureworkplace.com,0001-10-15 15:18:32,The Payment status has changed on the followin...,mark.mcconnell@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca...","[joe.stepenovitch@enron.com, tmartin@enron.com...","[richard.shapiro@enron.com, rick.buy@enron.com...","[n, o, r, e, ., c, m, @, a, i]","[richard.shapiro@enron.com, rick.buy@enron.com..."
2,201623,enron_update@concureworkplace.com,0001-10-15 15:18:32,The Approval status has changed on the followi...,mark.mcconnell@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca...","[joe.stepenovitch@enron.com, tmartin@enron.com...","[richard.shapiro@enron.com, rick.buy@enron.com...","[n, o, r, e, ., c, m, @, a, i]","[richard.shapiro@enron.com, rick.buy@enron.com..."
3,201624,enron_update@concureworkplace.com,0001-10-15 15:18:35,The Approval status has changed on the followi...,mark.mcconnell@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca...","[joe.stepenovitch@enron.com, tmartin@enron.com...","[richard.shapiro@enron.com, rick.buy@enron.com...","[n, o, r, e, ., c, m, @, a, i]","[richard.shapiro@enron.com, rick.buy@enron.com..."
4,82795,enron_update@concureworkplace.com,0001-10-17 13:06:55,The following expense report is ready for appr...,barry.tycholiz@enron.com,"[gbn@bus.utexas.edu, clay.harris@enron.com, ca...","[joe.stepenovitch@enron.com, tmartin@enron.com...","[richard.shapiro@enron.com, rick.buy@enron.com...","[n, o, r, e, ., c, m, @, a, i]","[richard.shapiro@enron.com, rick.buy@enron.com..."


## Bag of words with TF-IDF

In [79]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', stop_words = 'english', norm = 'l2')

In [80]:
from sklearn.metrics.pairwise import cosine_similarity

In [81]:
import time

In [82]:
def get_mids_sender_to_recipient(address_books, dataset_df):
    '''
    Computes the list of mids for each couple (sender, recipient)
    '''
    #a new dictionnary
    mids_sender_to_recipient = {}
    
    #all the senders
    all_senders = address_books_train.keys()
    
    for sender in tqdm(all_senders):
        #the mails sent by the sender
        mask_sender = (sender == dataset_df.sender)
        sub_dataset_df = dataset_df[mask_sender]
        sub_dataset_df = sub_dataset_df.reset_index(drop=True)
        
        rec_out = address_books[sender]
        for rec, out in rec_out: 

            #we search amongst them, those sent to the recipient
            mask_recipient = [rec in sub_dataset_df.recipients[i] for i in range(sub_dataset_df.shape[0])]

            #we save the result
            mids_sender_to_recipient[(sender, rec)] = sub_dataset_df['mid'][mask_recipient].values.tolist()
        
    return mids_sender_to_recipient 

In [83]:
mids_sender_to_recipient = get_mids_sender_to_recipient(address_books_train, training_df_train)

In [132]:
mids_sender_to_recipient['enron_update@concureworkplace.com', 'mark.whitt@enron.com'][:5]

['28043', '27895', '27898', '27985', '28076']

In [143]:
import nltk
import string
import os

from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def stem_tokens(tokens):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = stem_tokens(tokens)
    return stems

tfidf = TfidfVectorizer(tokenizer=tokenize, stop_words='english')

In [165]:
#early optimization is the root of all evil
k=10
#Make some predictions
training_df_test['tfidf_centroid_pred'] = None

for row in tqdm(range(training_df_test.shape[0])): #for each mail
    # Search for the sender
    sender = training_df_test['sender'][row]
    print(sender)
    
    #Search for all his previous emails sent (content + mid)
    corpus = list(training_df_train[training_df_train['sender'] == sender]['body']) #we may have some doublons
    print(len(corpus))
    #mids = emails_ids_per_sender_train[sender] #we have to keep them
    mids = list(training_df_train[training_df_train['sender'] == sender]['mid'])
    print(len(mids))
    
    #DO NOT : Add the new document in the test
    #corpus = docs # + [training_df_test['body'][row]]
    #print(len(documents))
    
    #Clean the documents
    corpus_clean = {}
    for i, doc in enumerate(corpus):
        lowers = doc.lower()
        #no_punctuation = lowers#.translate(string.punctuation)
        corpus_clean[mids[i]] = lowers

    # == TFIDF ====
    #corpus
    
    tfidf_matrix_sender = tfidf.fit_transform(corpus_clean.values())

    # new document
    new_doc = training_df_test['body'][row]
    response = tfidf.transform([new_doc])
    response
        
    #We search the centroids for each recipient of sender
    centroid_recipient = {} #contains for each recipient its vector representation

    kmin = min(20, len(address_books_train[sender]))
    for recipient, out in address_books_train[sender][:kmin]:
        mids_sender_recipient = mids_sender_to_recipient[sender, recipient]
        mask_mid = [mids[i] in list(mids_sender_recipient) for i in range(len(mids))]

        centroid_recipient[recipient] = np.sum(tfidf_matrix_sender.todense()[mask_mid], axis = 0)

    print(len(centroid_recipient)) 
    
    #We calcultate the score for each recipient (cosine similarity) 
    score_recipient = {}
    for recipient, centroid in centroid_recipient.items():
        score_recipient[recipient] = float(cosine_similarity(centroid,response)[0])
    
    sorted_score_recipient = sorted(score_recipient.items(), key=operator.itemgetter(1), reverse = True)
    
    #prediction
    training_df_test['tfidf_centroid_pred'][row] = [elt[0] for elt in sorted_score_recipient[:k]]
    
    break

  0%|                                                 | 0/8723 [00:00<?, ?it/s]

chris.germany@enron.com
1968
1968
20


C:\Users\Mehdi\Annexes\PythonENSAE_v2\python\lib\site-packages\numpy\matrixlib\defmatrix.py:318: FutureWarning: in the future, boolean array-likes will be handled as a boolean array index
  out = N.ndarray.__getitem__(self, index)


In [167]:
training_df_test.head(1)

,mid,sender,date,body,recipients,random_pred,freq_pred,weighted_freq_pred,name_freq_pred,tfidf_centroid_pred
0,337773,chris.germany@enron.com,2000-03-08 10:05:00,Looks good.Stephanie Sever 03/08/2000 05:35 ...,[stephanie.sever@enron.com],"[rudy.elizondo@enron.com, tony.harris@enron.co...","[scott.goodell@enron.com, victor.lamadrid@enro...","[scott.goodell@enron.com, jim.homco@enron.com,...","[scott.goodell@enron.com, victor.lamadrid@enro...","[scott.goodell@enron.com, victor.lamadrid@enro..."


In [ ]:
# ======== False

In [119]:
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()



In [133]:
#early optimization is the root of all evil
k=10
#Make some predictions
training_df_test['tfidf_centroid_pred'] = None

for row in tqdm(range(training_df_test.shape[0])): #for each mail
    # Search for the sender
    sender = training_df_test['sender'][row]
    print(sender)
    
    #Search for all his previous emails sent (content + mid)
    docs = list(training_df_train[training_df_train['sender'] == sender]['body']) #we may have some doublons
    print(len(docs))
    #mids = emails_ids_per_sender_train[sender] #we have to keep them
    mids = list(training_df_train[training_df_train['sender'] == sender]['mid'])
    print(len(mids))
    
    #Add the new document in the test
    documents = docs + [training_df_test['body'][row]]
    print(len(documents))
    
    #Lowerize the text
    for i, doc in enumerate(documents):
        documents[i] = doc.lower()
    
    #Stemming the whole corpus
    for text in documents:
        text_stem = []
        for word in text.split(' '):
            word_stem = porter_stemmer.stem(word)
            text_stem.append(word_stem) 
            text_stem2 = ' '.join(text_stem)
        break
    break
    
    '''
    We have forgetten to use stems !
    '''
    #We make a TF-IDF on all these documents
    tfidf_matrix_sender = tf.fit_transform(documents)
   
    #We search the centroids for each recipient of sender
    centroid_recipient = {} #contains for each recipient its vector representation

    for recipient, out in address_books_train[sender]:
        mids_sender_recipient = mids_sender_to_recipient[sender, recipient]
        
        mask_mid = [mids[i] in list(mids_sender_recipient) for i in range(len(mids))]
        mask_mid = mask_mid + [False] #for the new document

        centroid_recipient[recipient] = np.sum(tfidf_matrix_sender.todense()[mask_mid], axis = 0)

    print(len(centroid_recipient))    
    #We calcultate the score for each recipient (cosine similarity) 
    score_recipient = {}
    for recipient, centroid in centroid_recipient.items():
        score_recipient[recipient] = float(cosine_similarity(centroid,tfidf_matrix_sender.todense()[-1,:])[0])
    
    sorted_score_recipient = sorted(score_recipient.items(), key=operator.itemgetter(1), reverse = True)
    
    #prediction
    training_df_test['tfidf_centroid_pred'][row] = [elt[0] for elt in sorted_score_recipient[:k]]
    
    break

  0%|                                                 | 0/8723 [00:00<?, ?it/s]

chris.germany@enron.com
1968
1968
1969


# Test the P@10

In [35]:
training_df_test.head()

,mid,sender,date,body,recipients,random_pred,freq_pred,weighted_freq_pred
0,337773,chris.germany@enron.com,2000-03-08 10:05:00,Looks good.Stephanie Sever 03/08/2000 05:35 ...,[stephanie.sever@enron.com],"[rudy.elizondo@enron.com, tony.harris@enron.co...","[scott.goodell@enron.com, victor.lamadrid@enro...","[o, n, e, ., r, m, c, a, @, l]"
1,112428,susan.scott@enron.com,2001-05-04 12:02:26,"oh, it s on!","[adriane.schultea@enron.com, misti.day@enron.c...","[tbroderi@czn.com, antoine.pierre@enron.com, 9...","[jeffery.fawcett@enron.com, lorraine.lindberg@...","[o, e, n, ., m, c, r, a, l, @]"
2,362746,chris.dorland@enron.com,2000-11-13 03:25:00,George has Oyster Creek coming back Nov. 25. W...,[ricardo.perez@enron.com],"[stephen.wood@enron.com, jim.white@las-cruces....","[dan.dorland@enron.com, rlaird@natsource.ca, k...","[o, n, e, r, ., c, m, a, l, @]"
3,333249,c..giron@enron.com,2000-07-24 03:17:00,Last Friday I was trying to get into my accoun...,[onepass@coair.com],"[us@enron.com, patrick.mulvany@enron.com, vire...","[carole.frank@enron.com, kristi.giron@cfisd.ne...","[n, o, r, e, ., c, m, i, a, @]"
4,384123,eric.bass@enron.com,2000-08-17 06:53:00,"I want to trade you a 4th, 6th and 8th round p...",[lqcolombo@aol.com],"[cheryl.dudley@enron.com, diana.scholtes@enron...","[bryan.hull@enron.com, matthew.lenhart@enron.c...","[n, o, e, ., a, r, m, c, @, s]"


In [28]:
#Split the recipients to use kdd_mapk
for row in range(training_df_test.shape[0]):
    training_df_test['recipients'][row] = training_df_test['recipients'][row].split(' ')

In [29]:
# MAP@10 for random predictions
mapk(training_df_test['recipients'], training_df_test['random_pred'])

0.00061432874957465129

In [53]:
# MAP@10 for random predictions among the address book of each sender
mapk(training_df_test['recipients'], training_df_test['random_pred_address_book'])

0.034742122198211191

In [30]:
# MAP@10 for frequency-based predictions
mapk(training_df_test['recipients'], training_df_test['freq_pred'])

0.28947520561737972

In [61]:
# MAP@10 for weighted-frequency-based predictions
mapk(training_df_test['recipients'], training_df_test['weighted_freq_pred'])

0.28388610397538971

In [78]:
# MAP@10 for name_frequency-based predictions
mapk(training_df_test['recipients'], training_df_test['name_freq_pred'])

0.29334070469169998

In [56]:
# MAP@10 for TF-IDF centroids predictions (withour STEM ! for the moment)
mapk(training_df_test['recipients'], training_df_test['tfidf_centroid_pred'])

TypeError: object of type 'NoneType' has no len()

Let's check that in the dataframe :

In [57]:
#Score for each prediction with random recommendation
training_df_test['score_random'] = None

for row in range(training_df_test.shape[0]): #for each mail
    training_df_test['score_random'][row] = apk(training_df_test['recipients'][row], training_df_test['random_pred'][row], k=10)

In [58]:
#Score for each prediction with frequency-based recommendations 
training_df_test['score_freq'] = None

for row in range(training_df_test.shape[0]): #for each mail
    training_df_test['score_freq'][row] = apk(training_df_test['recipients'][row], training_df_test['freq_pred'][row], k=10)    

# Application on the real test dataset

In [59]:
test_df = transform_dataset(test_info, test_set)

In [60]:
test_df.head()

,mid,sender,date,body
0,284098,jonathan.mckay@enron.com,2001-11-02 05:25:29,"How is everyone.....mother, child.........fath..."
1,272008,dutch.quigley@enron.com,2001-11-02 05:34:55,-----Original Message-----From: \tWesner-Soon...
2,49273,james.d.steffes@enron.com,2001-11-02 05:57:55,Janine -Ok for you to cover the whole country....
3,71901,kim.ward@enron.com,2001-11-02 06:10:47,when?
4,82354,barry.tycholiz@enron.com,2001-11-02 06:17:44,WOW.... I am positive that your beautiful wife...


# Write 

In [62]:
#Prediction according to the frequency and the name (first word in the mail)


with open(path_to_results + 'predictions_name_freq_.txt', 'wb') as my_file:
    my_file.write(bytes('mid,recipients' + '\n', 'UTF-8'))
    for row in range(test_df.shape[0]):
        mid = test_df['mid'][row]
        name_freq_preds = test_df['name_freq_pred'][row]
        my_file.write(bytes(str(mid) + ',' + ' '.join(name_freq_preds) + '\n', 'UTF-8'))

KeyError: 'name_freq_pred'